# Section: Federated Learning

# Lesson: Introducing Federated Learning

Federated Learning is a technique for training Deep Learning models on data to which you do not have access. Basically:

Federated Learning: Instead of bringing all the data to one machine and training a model, we bring the model to the data, train it locally, and merely upload "model updates" to a central server.

Use Cases:

    - app company (Texting prediction app)
    - predictive maintenance (automobiles / industrial engines)
    - wearable medical devices
    - ad blockers / autotomplete in browsers (Firefox/Brave)
    
Challenge Description: data is distributed amongst sources but we cannot aggregated it because of:

    - privacy concerns: legal, user discomfort, competitive dynamics
    - engineering: the bandwidth/storage requirements of aggregating the larger dataset

# Lesson: Introducing / Installing PySyft

In order to perform Federated Learning, we need to be able to use Deep Learning techniques on remote machines. This will require a new set of tools. Specifically, we will use an extensin of PyTorch called PySyft.

### Install PySyft

Hi, Run the cell below and it will automatic install all the dependecies for PySyft. Use this Google Colab Workspace for your use 

Thank You

If you come across any issues, then feel free to contact me on Slack. My username is archit

In [1]:
!pip install tf-encrypted

! URL="https://github.com/openmined/PySyft.git" && FOLDER="PySyft" && if [ ! -d $FOLDER ]; then git clone -b dev --single-branch $URL; else (cd $FOLDER && git pull $URL && cd ..); fi;

!cd PySyft; python setup.py install  > /dev/null

import os
import sys
module_path = os.path.abspath(os.path.join('./PySyft'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
!pip install --upgrade --force-reinstall lz4
!pip install --upgrade --force-reinstall websocket
!pip install --upgrade --force-reinstall websockets
!pip install --upgrade --force-reinstall zstd

     |████████████████████████████████| 2.1MB 4.8MB/s 
     |████████████████████████████████| 266kB 44.6MB/s 
  Stored in directory: /root/.cache/pip/wheels/d9/45/dd/65f0b38450c47cf7e5312883deb97d065e030c5cca0a365030
Successfully built pyyaml
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Cloning into 'PySyft'...
remote: Enumerating objects: 30034, done.
remote: Total 30034 (delta 0), reused 0 (delta 0), pack-reused 30034
Receiving objects: 100% (30034/30034), 32.86 MiB | 17.98 MiB/s, done.
Resolving deltas: 100% (20118/20118), done.
zip_safe flag not set; analyzing archive contents...
zip_safe flag not set; analyzing archive contents...
__pycache__.zstd.cpython-36: module references __file__
     |████████████████████████████████| 389kB 4.8MB/s 
ERROR: syft 0.1.21a1 has requirement msgpack>=0.6.1, but you'll have msgpack 0.5.6 which is incompatible.
ERROR: syft 0.1.21a1 has requirement tf_encrypted!=0.5.7,>=0.5.4, b

In [0]:
import torch as th

In [3]:
x = th.tensor([1,2,3,4,5])
x

tensor([1, 2, 3, 4, 5])

In [0]:
y = x + x

In [5]:
print(y)

tensor([ 2,  4,  6,  8, 10])


In [6]:
import syft as sy

W0801 03:54:37.115846 140255265081216 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was '/usr/local/lib/python3.6/dist-packages/tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0801 03:54:37.132336 140255265081216 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/tf_encrypted/session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



In [0]:
hook = sy.TorchHook(th)

In [8]:
th.tensor([1,2,3,4,5])

tensor([1, 2, 3, 4, 5])

# Lesson: Basic Remote Execution in PySyft

## PySyft => Remote PyTorch

The essence of Federated Learning is the ability to train models in parallel on a wide number of machines. Thus, we need the ability to tell remote machines to execute the operations required for Deep Learning.

Thus, instead of using Torch tensors - we're now going to work with **pointers** to tensors. Let me show you what I mean. First, let's create a "pretend" machine owned by a "pretend" person - we'll call him Bob.

In [0]:
bob = sy.VirtualWorker(hook, id="bob")

In [10]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x = x.send(bob)

In [13]:
bob._objects

{83546248136: tensor([1, 2, 3, 4, 5])}

In [14]:
x.location

<VirtualWorker id:bob #objects:1>

In [15]:
x.id_at_location

83546248136

In [16]:
x.id

83796008281

In [17]:
x.owner

<VirtualWorker id:me #objects:0>

In [18]:
hook.local_worker

<VirtualWorker id:me #objects:0>

In [19]:
x

(Wrapper)>[PointerTensor | me:83796008281 -> bob:83546248136]

In [20]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [21]:
bob._objects

{}

# Project: Playing with Remote Tensors

In this project, I want you to .send() and .get() a tensor to TWO workers by calling .send(bob,alice). This will first require the creation of another VirtualWorker called alice.

In [0]:
alice = sy.VirtualWorker(hook, id="alice")

In [0]:
x = th.tensor([1,2,3,4,5])

In [0]:
x_ptr = x.send(bob, alice)

In [25]:
x_ptr.child.child

{'alice': [PointerTensor | me:33902331941 -> alice:61454519900],
 'bob': [PointerTensor | me:9994193383 -> bob:62164698419]}

In [26]:
x_ptr

(Wrapper)>[MultiPointerTensor]
	-> [PointerTensor | me:9994193383 -> bob:62164698419]
	-> [PointerTensor | me:33902331941 -> alice:61454519900]

# Lesson: Introducing Remote Arithmetic

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(bob)

In [28]:
x

(Wrapper)>[PointerTensor | me:92259322464 -> bob:60300387529]

In [29]:
y

(Wrapper)>[PointerTensor | me:34122837177 -> bob:22803051715]

In [0]:
z = x + y

In [31]:
z

(Wrapper)>[PointerTensor | me:79488974391 -> bob:28676291638]

In [32]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [33]:
z = th.add(x,y)
z

(Wrapper)>[PointerTensor | me:72511455007 -> bob:68321991695]

In [34]:
z = z.get()
z

tensor([2, 3, 4, 5, 6])

In [0]:
x = th.tensor([1.,2,3,4,5], requires_grad=True).send(bob)
y = th.tensor([1.,1,1,1,1], requires_grad=True).send(bob)

In [0]:
z = (x + y).sum()

In [37]:
z.backward()

(Wrapper)>[PointerTensor | me:21134382454 -> bob:83378135776]

In [0]:
x = x.get()

In [39]:
x

tensor([1., 2., 3., 4., 5.], requires_grad=True)

In [40]:
x.grad

tensor([1., 1., 1., 1., 1.])

# Project: Learn a Simple Linear Model

In this project, I'd like for you to create a simple linear model which will solve for the following dataset below. You should use only Variables and .backward() to do so (no optimizers or nn.Modules). Furthermore, you must do so with both the data and the model being located on Bob's machine.

In [0]:
input = th.tensor([[1.,1],[0,1,],[1,0],[0,0]], requires_grad=True).send(bob)
target = th.tensor([[1.],[1],[0],[0]], requires_grad=True).send(bob)

In [0]:
weights = th.tensor([[0.],[0.]], requires_grad=True).send(bob)

In [43]:
for i in range(10):
    
    pred = input.mm(weights)
    
    loss = ((pred - target)**2).sum()
    
    loss.backward()
    
    weights.data.sub_(weights.grad * 0.1)
    weights.grad *= 0
    
    print(loss.get().data)

tensor(2.)
tensor(0.5600)
tensor(0.2432)
tensor(0.1372)
tensor(0.0849)
tensor(0.0538)
tensor(0.0344)
tensor(0.0220)
tensor(0.0141)
tensor(0.0090)


# Lesson: Garbage Collection and Common Errors


In [0]:
bob = bob.clear_objects()

In [45]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [47]:
bob._objects

{96602211010: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [49]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [51]:
bob._objects

{70474704424: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [53]:
bob._objects

{}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [55]:
x

(Wrapper)>[PointerTensor | me:64807980933 -> bob:41504340880]

In [56]:
bob._objects

{41504340880: tensor([1, 2, 3, 4, 5])}

In [0]:
x = "asdf"

In [58]:
bob._objects

{41504340880: tensor([1, 2, 3, 4, 5])}

In [0]:
del x

In [60]:
bob._objects

{41504340880: tensor([1, 2, 3, 4, 5])}

In [61]:
bob = bob.clear_objects()
bob._objects

{}

In [0]:
for i in range(1000):
    x = th.tensor([1,2,3,4,5]).send(bob)

In [63]:
bob._objects

{60991015607: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1])

In [65]:
z = x + y

TensorsNotCollocatedException: ignored

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)
y = th.tensor([1,1,1,1,1]).send(alice)

In [67]:
z = x + y

TensorsNotCollocatedException: ignored

# Lesson: Toy Federated Learning

Let's start by training a toy model the centralized way. This is about a simple as models get. We first need:

- a toy dataset
- a model
- some basic training logic for training a model to fit the data.

In [0]:
from torch import nn, optim

In [0]:
# A Toy Dataset
data = th.tensor([[1.,1],[0,1],[1,0],[0,0]], requires_grad=True)
target = th.tensor([[1.],[1], [0], [0]], requires_grad=True)

In [0]:
# A Toy Model
model = nn.Linear(2,1)

In [0]:
opt = optim.SGD(params=model.parameters(), lr=0.1)

In [72]:
def train(iterations=20):
    for iter in range(iterations):
        opt.zero_grad()

        pred = model(data)

        loss = ((pred - target)**2).sum()

        loss.backward()

        opt.step()

        print(loss.data)
        
train()

tensor(2.4351)
tensor(1.1778)
tensor(0.7976)
tensor(0.5692)
tensor(0.4111)
tensor(0.2991)
tensor(0.2189)
tensor(0.1611)
tensor(0.1192)
tensor(0.0886)
tensor(0.0661)
tensor(0.0495)
tensor(0.0372)
tensor(0.0280)
tensor(0.0211)
tensor(0.0160)
tensor(0.0121)
tensor(0.0092)
tensor(0.0070)
tensor(0.0053)


In [0]:
data_bob = data[0:2].send(bob)
target_bob = target[0:2].send(bob)

In [0]:
data_alice = data[2:4].send(alice)
target_alice = target[2:4].send(alice)

In [0]:
datasets = [(data_bob, target_bob), (data_alice, target_alice)]

In [0]:
def train(iterations=20):

    model = nn.Linear(2,1)
    opt = optim.SGD(params=model.parameters(), lr=0.1)
    
    for iter in range(iterations):

        for _data, _target in datasets:

            # send model to the data
            model = model.send(_data.location)

            # do normal training
            opt.zero_grad()
            pred = model(_data)
            loss = ((pred - _target)**2).sum()
            loss.backward()
            opt.step()

            # get smarter model back
            model = model.get()

            print(loss.get())

In [77]:
train()

tensor(0.3609, requires_grad=True)
tensor(2.3101, requires_grad=True)
tensor(0.7819, requires_grad=True)
tensor(1.2887, requires_grad=True)
tensor(0.4895, requires_grad=True)
tensor(0.7622, requires_grad=True)
tensor(0.2866, requires_grad=True)
tensor(0.4546, requires_grad=True)
tensor(0.1675, requires_grad=True)
tensor(0.2725, requires_grad=True)
tensor(0.0983, requires_grad=True)
tensor(0.1643, requires_grad=True)
tensor(0.0580, requires_grad=True)
tensor(0.0998, requires_grad=True)
tensor(0.0345, requires_grad=True)
tensor(0.0610, requires_grad=True)
tensor(0.0207, requires_grad=True)
tensor(0.0377, requires_grad=True)
tensor(0.0125, requires_grad=True)
tensor(0.0235, requires_grad=True)
tensor(0.0077, requires_grad=True)
tensor(0.0148, requires_grad=True)
tensor(0.0048, requires_grad=True)
tensor(0.0094, requires_grad=True)
tensor(0.0031, requires_grad=True)
tensor(0.0061, requires_grad=True)
tensor(0.0020, requires_grad=True)
tensor(0.0040, requires_grad=True)
tensor(0.0013, requi

# Lesson: Advanced Remote Execution Tools

In the last section we trained a toy model using Federated Learning. We did this by calling .send() and .get() on our model, sending it to the location of training data, updating it, and then bringing it back. However, at the end of the example we realized that we needed to go a bit further to protect people privacy. Namely, we want to average the gradients BEFORE calling .get(). That way, we won't ever see anyone's exact gradient (thus better protecting their privacy!!!)

But, in order to do this, we need a few more pieces:

- use a pointer to send a Tensor directly to another worker

And in addition, while we're here, we're going to learn about a few more advanced tensor operations as well which will help us both with this example and a few in the future!

In [79]:
bob.clear_objects()


<VirtualWorker id:bob #objects:0>

In [80]:
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [0]:
x = x.send(alice)

In [83]:
bob._objects

{62339802862: tensor([1, 2, 3, 4, 5])}

In [84]:
alice._objects

{59104978067: (Wrapper)>[PointerTensor | alice:59104978067 -> bob:62339802862]}

In [0]:
y = x + x

In [86]:
y

(Wrapper)>[PointerTensor | me:80294123507 -> alice:32991947120]

In [87]:
bob._objects

{40358814342: tensor([ 2,  4,  6,  8, 10]),
 62339802862: tensor([1, 2, 3, 4, 5])}

In [88]:
alice._objects

{32991947120: (Wrapper)>[PointerTensor | alice:32991947120 -> bob:40358814342],
 59104978067: (Wrapper)>[PointerTensor | alice:59104978067 -> bob:62339802862]}

In [0]:
jon = sy.VirtualWorker(hook, id="jon")

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [91]:
bob._objects

{77785890472: tensor([1, 2, 3, 4, 5])}

In [92]:
alice._objects

{16756069860: (Wrapper)>[PointerTensor | alice:16756069860 -> bob:77785890472]}

In [93]:
x = x.get()
x

(Wrapper)>[PointerTensor | me:16756069860 -> bob:77785890472]

In [94]:
bob._objects

{77785890472: tensor([1, 2, 3, 4, 5])}

In [95]:
alice._objects

{}

In [96]:
x = x.get()
x

tensor([1, 2, 3, 4, 5])

In [97]:
bob._objects

{}

In [0]:
bob.clear_objects()
alice.clear_objects()

x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [99]:
bob._objects

{72080328582: tensor([1, 2, 3, 4, 5])}

In [100]:
alice._objects

{4201321661: (Wrapper)>[PointerTensor | alice:4201321661 -> bob:72080328582]}

In [0]:
del x

In [102]:
bob._objects

{}

In [103]:
alice._objects

{}

# Lesson: Pointer Chain Operations

In [104]:
bob.clear_objects()
alice.clear_objects()

<VirtualWorker id:alice #objects:0>

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob)

In [106]:
bob._objects

{38986996332: tensor([1, 2, 3, 4, 5])}

In [107]:
alice._objects

{}

In [108]:
x.move(alice)

(Wrapper)>[PointerTensor | me:81314629153 -> alice:95521226399]

In [109]:
bob._objects

{}

In [110]:
alice._objects

{95521226399: tensor([1, 2, 3, 4, 5])}

In [0]:
x = th.tensor([1,2,3,4,5]).send(bob).send(alice)

In [112]:
bob._objects

{789214019: tensor([1, 2, 3, 4, 5])}

In [113]:
alice._objects

{12447816204: (Wrapper)>[PointerTensor | alice:12447816204 -> bob:789214019],
 95521226399: tensor([1, 2, 3, 4, 5])}

In [114]:
x.remote_get()

(Wrapper)>[PointerTensor | me:58625476724 -> alice:12447816204]

In [115]:
bob._objects

{}

In [116]:
alice._objects

{12447816204: tensor([1, 2, 3, 4, 5]), 95521226399: tensor([1, 2, 3, 4, 5])}

In [117]:
x.move(bob)

(Wrapper)>[PointerTensor | me:58295915638 -> bob:58625476724]

In [118]:
x

(Wrapper)>[PointerTensor | me:58295915638 -> bob:58625476724]

In [119]:
bob._objects

{58625476724: tensor([1, 2, 3, 4, 5])}

In [120]:
alice._objects

{95521226399: tensor([1, 2, 3, 4, 5])}